<a href="https://colab.research.google.com/github/amedyukhina/biomassters/blob/main/data_loading_sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile requirements.txt

pandas==1.3.5
scikit-learn==1.0.2
tqdm==4.64.0
numpy==1.21.6
torch==1.12.1
torchvision==0.13.1
scikit-image==0.18.3
matplotlib==3.2.2
rasterio==1.2.0
boto3==1.26.16

Overwriting requirements.txt


In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
import os
import rasterio

In [4]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
path = 'gdrive/MyDrive/biomassters/data/train_features/'

In [6]:
files = os.listdir(path)
files = [fn for fn in files if 'S2' in fn] 

In [7]:
files = files[:10] * 100

In [8]:
len(files)

1000

In [9]:
%%time
for fn in files:
  with rasterio.open(os.path.join(path, fn)) as src:
    img = src.read()

/usr/local/lib/python3.7/dist-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


CPU times: user 7.59 s, sys: 1.46 s, total: 9.06 s
Wall time: 15.8 s


In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('gdrive/MyDrive/biomassters/info/features_metadata.csv')

In [63]:
bucket_name = 'drivendata-competition-biomassters-public-us'

In [103]:
files = df['s3path_us'].values
files = [fn.replace(rf's3://{bucket_name}/', '') 
for fn in files[:100] if 'S2' in fn]

In [104]:
files = files[:10] * 100

In [105]:
os.environ["AWS_NO_SIGN_REQUEST"] = 'YES'

In [106]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config

In [107]:
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [108]:
files[0]

'train_features/0003d2eb_S2_00.tif'

In [109]:
# %%time
# for fn in files:
#   os.makedirs(os.path.dirname(fn), exist_ok=True)
#   s3.download_file(bucket_name, fn, fn)
#   with rasterio.open(fn) as src:
#     img = src.read()

In [110]:
from cachetools import cached, TTLCache

In [111]:
cache = TTLCache(maxsize=1000, ttl=86400)

In [112]:
@cached(cache)
def get_image(fn):
  obj = s3.get_object(Bucket=bucket_name, Key=fn)
  with rasterio.open(obj['Body']) as src:
    img = src.read()
  return img

In [113]:
%%time
for fn in files:
  img = get_image(fn)

/usr/local/lib/python3.7/dist-packages/rasterio/io.py:130: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  return DatasetReader(mempath, driver=driver, sharing=sharing, **kwargs)


CPU times: user 131 ms, sys: 25.1 ms, total: 156 ms
Wall time: 760 ms
